# Fake News Classifier using LSTM https://www.kaggle.com/c/fake-news/data

In [2]:
import pandas as pd


In [8]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: 

In [9]:
import pandas as pd

In [12]:
df = pd.read_csv('/content/contentdrive/MyDrive/data/train.csv')

In [13]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df = df.dropna()

In [16]:
X = df.drop('label',axis=1)

In [17]:
y=df['label']

In [18]:
X.shape

(18285, 4)

In [19]:
y.shape

(18285,)

In [20]:
import tensorflow as tf

In [21]:
tf.__version__

'2.15.0'

In [22]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [24]:
### vocab size
voc_size=5000

# One hot representation

In [25]:
messages =X.copy()

In [26]:
messages.reset_index(inplace=True)

In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
### dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    try:
        review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
        review=review.lower()
        review=review.split()
        review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
        review=' '.join(review)
        corpus.append(review)
    except KeyError:
        print(f"KeyError: Unable to find key {i} in the 'messages' DataFrame.")


In [30]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [31]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[1]

[919, 4283, 2443, 4445, 2955, 2778, 4648]

# embedding layer


In [32]:
sent_len=20;
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 2925  750 4096]
 [   0    0    0 ... 2955 2778 4648]
 [   0    0    0 ... 1506 4465  248]
 ...
 [   0    0    0 ...  913 1031 2061]
 [   0    0    0 ... 4764 3890 1396]
 [   0    0    0 ...  581 3016 4723]]


In [33]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  919, 4283, 2443, 4445, 2955, 2778, 4648], dtype=int32)

each word here given by index (eg 1488 ) will be conv to vector of 40 features

# MODEL

In [34]:
emb_vector_feat=40
model=Sequential()
model.add(Embedding(voc_size,emb_vector_feat,input_length=sent_len))
model.add(LSTM(100))
#100 neurons
model.add(Dense(1,activation='sigmoid')) # dense layer because it is a classification thing
#binary output
model.compile( loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


emb layer 20 inp ,40 out
lstm layer 100 neuron

In [35]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X_final,y_final,test_size=0.33,random_state=42)

# Model training


In [37]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 25ms/step - loss: 0.3363 - accuracy: 0.8453 - val_loss: 0.1984 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1407 - accuracy: 0.9435 - val_loss: 0.2068 - val_accuracy: 0.9074
Epoch 3/10
192/192 [==============================] - 5s 25ms/step - loss: 0.1087 - accuracy: 0.9587 - val_loss: 0.2409 - val_accuracy: 0.9067
Epoch 4/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0750 - accuracy: 0.9714 - val_loss: 0.2661 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0501 - accuracy: 0.9832 - val_loss: 0.3017 - val_accuracy: 0.9125
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0339 - accuracy: 0.9893 - val_loss: 0.3409 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0244 - accuracy: 0.9925 - val_loss: 0.3295 - val_accuracy: 0.9087

# Predict Data

In [38]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [39]:
y_pred=np.where(y_pred>0.5,1 ,0)
#1 => fake if prob >0.5

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 252, 2364]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9082021541010771

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

